In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
!wget https://users.dcc.uchile.cl/~mtoro/cursos/BD2020/adult_with_pii.csv

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.2 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,467 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubunt

In [ ]:
%%sql
DROP TABLE IF EXISTS person;
CREATE TABLE person
(
  name varchar(255),
  dob date,
  ssn  varchar(50),
  zip integer,
  age integer,
  workclass varchar(255),
  fnlwgt integer,
  education varchar(255),
  education_num integer,
  marital_status varchar(255),
  occupation varchar(255),
  relationship varchar(255),
  race varchar(255),
  sex varchar(50),
  capital_gain integer,
  capital_loss integer,
  hours_per_week integer,
  country varchar(255),
  target varchar(255)
);
ALTER TABLE ONLY person ADD CONSTRAINT "person_key" PRIMARY KEY (name,ssn);

 * postgresql+psycopg2://@/postgres


""


Suponga que el gobierno de Estados Unidos cuenta con un base de datos de resultados del censo (la información de identificación personal (PII) es inventada)

In [ ]:
%%sql
copy person(name, dob, ssn, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target) from
'/content/adult_with_pii.csv' delimiter ',' csv header;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql SELECT * FROM person limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga que los identificadores explícitos de las personas son las columnas name y ssn (social security number).
Para proteger la privacidad de las personas, el gobierno de-identifica esta tabla borrando las columnas name y ssn.

In [ ]:
%%sql
create view person_deid as select dob, zip, age, workclass, fnlwgt, education, education_num, marital_status, occupation, relationship, race, sex, capital_gain, capital_loss, hours_per_week, country, target from person;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from person_deid limit 5;

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,1967-09-07,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,1988-06-07,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,1991-08-06,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,2009-04-06,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,1951-09-16,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Suponga también que se filtró de una casa comercial, los nombres, fechas de nacimiento, ssn, código postal y edad de muchas personas. Nadie se preocupa por esto ya que no contiene información sensible (como el sueldo).
Crearemos una vista para representar esta data auxiliar.

In [ ]:
%%sql
create view aux as select name, dob, ssn, zip, age from person;

 * postgresql+psycopg2://@/postgres


""


In [ ]:
%sql select * from aux limit 5;

 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38
3,Dorry Poter,2009-04-06,659-57-4974,25503,53
4,Dick Honnan,1951-09-16,220-93-3811,75387,28


Para el lab, tenga en cuenta que puede guardar resultados de consultas en variables. Por ejemplo:

In [ ]:
r = %sql select * from aux where name='Karrie Trusslove';
#podemos usar r como un dataframe
print("cantidad de registros:",len(r))
#usamos ['age'] para projectar la columna 'age', e iloc[0] para acceder a la fila número 0.
print("Edad de Karrie:", r['age'].iloc[0])
#puede construir consultas dinámicas usando interpolación como se describe a continuación:
def foo(tabla):
  r = %sql select * from $tabla
  return r
foo('person')

 * postgresql+psycopg2://@/postgres
cantidad de registros: 1
Edad de Karrie: 39
 * postgresql+psycopg2://@/postgres


,name,dob,ssn,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,target
0,Karrie Trusslove,1967-09-07,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,1988-06-07,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,1991-08-06,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,2009-04-06,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,1951-09-16,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,Ardyce Golby,1961-10-29,212-61-8338,41328,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,Jean O'Connor,1952-06-28,737-32-2919,94735,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,Reuben Skrzynski,1966-08-09,314-48-0219,49628,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,Caye Biddle,1978-05-19,647-75-3550,8213,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


#Pregunta 1

Escriba una función que tome como argumento un conjunto de columnas y que realice un ataque de asociación a la tabla person_deid usando la tabla aux.
Muestre el resultado de al menos 3 ataques de asociación.


In [ ]:
def ataque_asociacion(cols: list[str]):
  col1 = cols[0]
  col2 = cols[1]
  r = %sql select * from person_deid p join aux a on p.$col1 = a.$col1 and p.$col2 = a.$col2
  return r
cols1 = ["dob", "age"]
cols2 = ["dob", "zip"]
cols3 = ["zip", "age"]
ataque_asociacion(cols1)

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,...,capital_gain,capital_loss,hours_per_week,country,target,name,dob,ssn,zip,age
0,1950-09-07,77938,31,Self-emp-not-inc,30290,HS-grad,9,Never-married,Other-service,Unmarried,...,0,0,40,United-States,<=50K,Katheryn Staker,1950-09-07,598-12-8351,77938,31
1,1950-09-07,56390,45,Private,102147,Assoc-voc,11,Divorced,Exec-managerial,Not-in-family,...,0,0,40,United-States,>50K,Alexio Pendered,1950-09-07,563-69-1278,56390,45
2,1950-09-08,66865,21,Private,169699,Some-college,10,Never-married,Other-service,Not-in-family,...,0,0,35,United-States,<=50K,Tanitansy Kenway,1950-09-08,413-21-3985,66865,21
3,1950-09-08,78217,61,Private,99784,Masters,14,Widowed,Prof-specialty,Not-in-family,...,0,0,40,United-States,>50K,Enrika Robus,1950-09-08,851-15-9861,78217,61
4,1950-09-09,2037,31,Private,219318,HS-grad,9,Never-married,Adm-clerical,Unmarried,...,0,0,35,Puerto-Rico,<=50K,Wain McPake,1950-09-09,178-20-0160,2037,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33578,2012-09-02,8036,58,Private,114495,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,...,0,0,40,United-States,<=50K,Debi McFadden,2012-09-02,715-81-6122,8036,58
33579,2012-09-03,95172,45,Private,179428,Some-college,10,Married-civ-spouse,Transport-moving,Husband,...,0,0,55,United-States,<=50K,Edith Hurne,2012-09-03,511-49-7897,95172,45
33580,2012-09-04,32883,31,Self-emp-not-inc,323020,Some-college,10,Married-civ-spouse,Other-service,Wife,...,0,0,30,United-States,<=50K,Fancy Thomassen,2012-09-04,183-16-1127,32883,31
33581,2012-09-05,13614,36,Self-emp-inc,108293,Bachelors,13,Married-civ-spouse,Exec-managerial,Wife,...,0,1977,45,United-States,>50K,Monroe McEvilly,2012-09-05,411-87-2432,13614,36


In [ ]:
ataque_asociacion(cols2)

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,...,capital_gain,capital_loss,hours_per_week,country,target,name,dob,ssn,zip,age
0,1950-09-07,56390,45,Private,102147,Assoc-voc,11,Divorced,Exec-managerial,Not-in-family,...,0,0,40,United-States,>50K,Alexio Pendered,1950-09-07,563-69-1278,56390,45
1,1950-09-07,77938,31,Self-emp-not-inc,30290,HS-grad,9,Never-married,Other-service,Unmarried,...,0,0,40,United-States,<=50K,Katheryn Staker,1950-09-07,598-12-8351,77938,31
2,1950-09-08,66865,21,Private,169699,Some-college,10,Never-married,Other-service,Not-in-family,...,0,0,35,United-States,<=50K,Tanitansy Kenway,1950-09-08,413-21-3985,66865,21
3,1950-09-08,78217,61,Private,99784,Masters,14,Widowed,Prof-specialty,Not-in-family,...,0,0,40,United-States,>50K,Enrika Robus,1950-09-08,851-15-9861,78217,61
4,1950-09-09,2037,31,Private,219318,HS-grad,9,Never-married,Adm-clerical,Unmarried,...,0,0,35,Puerto-Rico,<=50K,Wain McPake,1950-09-09,178-20-0160,2037,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32558,2012-09-02,70373,56,Private,371064,9th,5,Married-civ-spouse,Other-service,Husband,...,0,0,40,United-States,<=50K,Mervin Tales,2012-09-02,330-42-6031,70373,56
32559,2012-09-03,95172,45,Private,179428,Some-college,10,Married-civ-spouse,Transport-moving,Husband,...,0,0,55,United-States,<=50K,Edith Hurne,2012-09-03,511-49-7897,95172,45
32560,2012-09-04,32883,31,Self-emp-not-inc,323020,Some-college,10,Married-civ-spouse,Other-service,Wife,...,0,0,30,United-States,<=50K,Fancy Thomassen,2012-09-04,183-16-1127,32883,31
32561,2012-09-05,13614,36,Self-emp-inc,108293,Bachelors,13,Married-civ-spouse,Exec-managerial,Wife,...,0,1977,45,United-States,>50K,Monroe McEvilly,2012-09-05,411-87-2432,13614,36


In [ ]:
ataque_asociacion(cols3)

 * postgresql+psycopg2://@/postgres


,dob,zip,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,...,capital_gain,capital_loss,hours_per_week,country,target,name,dob,ssn,zip,age
0,1951-09-02,4,55,Private,145214,HS-grad,9,Divorced,Adm-clerical,Not-in-family,...,0,0,40,United-States,<=50K,Brandise Lynskey,1951-09-02,572-18-6292,4,55
1,1977-11-23,12,24,Private,508548,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,...,0,0,50,United-States,<=50K,Helenka Jovasevic,1977-11-23,828-68-8139,12,24
2,1985-02-09,16,59,None,87247,10th,6,Divorced,None,Not-in-family,...,0,0,40,England,<=50K,Nicko Zavattari,1985-02-09,119-87-0848,16,59
3,1985-07-18,17,32,Private,118161,HS-grad,9,Married-civ-spouse,Protective-serv,Husband,...,0,0,40,United-States,<=50K,Willyt Newvill,1985-07-18,220-93-2868,17,32
4,1999-02-26,17,52,Self-emp-not-inc,135339,Some-college,10,Married-civ-spouse,Craft-repair,Husband,...,0,0,40,None,>50K,Blondie Capes,1999-02-26,306-84-6162,17,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32750,1986-03-16,99989,18,Private,89760,12th,8,Never-married,Other-service,Own-child,...,0,0,20,United-States,<=50K,Cosimo Benoix,1986-03-16,161-66-8547,99989,18
32751,1972-03-08,99989,20,Private,231231,Some-college,10,Never-married,Sales,Not-in-family,...,0,0,30,United-States,<=50K,Rita Paridge,1972-03-08,554-71-0791,99989,20
32752,1986-03-11,99991,17,Private,152652,11th,7,Never-married,Handlers-cleaners,Own-child,...,0,0,25,United-States,<=50K,Adele Wallbank,1986-03-11,346-57-6278,99991,17
32753,1971-03-04,99995,31,Private,159187,Some-college,10,Married-civ-spouse,Craft-repair,Husband,...,0,0,40,United-States,>50K,Nicoli Godain,1971-03-04,120-52-6209,99995,31


#  Pregunta 2

1.   Escriba cuántos individuos en esta base de datos son únicamente identificados por el código postal (zip). ¿A qué porcentaje del total corresponde? Recuerde que una persona es identificada únicamente por la combinación nombre y ssn.

In [ ]:
zips = %sql select count(c) as zips_unicos from (select count(*) as cuenta from person group by zip having count(zip) = 1) as c
personas = %sql select count(distinct (name, ssn)) as personas from person
zips_unicos = zips['zips_unicos'].iloc[0]
personas_unicas = personas['personas'].iloc[0]
print("Personas identificadas únicamente por el zip:", zips_unicos)
print("Total de personas:", personas_unicas)
print("Porcentaje respecto del total: {0:0.2f}%".format(zips_unicos/personas_unicas*100))

 * postgresql+psycopg2://@/postgres
 * postgresql+psycopg2://@/postgres
Personas identificadas únicamente por el zip: 23513
Total de personas: 32561
Porcentaje respecto del total: 72.21%


2.   Haga lo mismo para el CId edad (age).

In [ ]:
ages = %sql select count(c) as edades_unicas from (select count(*) as cuenta from person group by age having count(age) = 1) as c
personas = %sql select count(distinct (name, ssn)) as personas from person
edades_unicas = ages['edades_unicas'].iloc[0]
print("Personas identificadas únicamente por el zip:", edades_unicas)
print("Total de personas:", personas_unicas)
print("Porcentaje respecto del total: {0:0.2f}%".format(edades_unicas/personas_unicas*100))

 * postgresql+psycopg2://@/postgres
 * postgresql+psycopg2://@/postgres
Personas identificadas únicamente por el zip: 2
Total de personas: 32561
Porcentaje respecto del total: 0.01%


3.   ¿Qué puede concluir de este análisis?

Podemos notar que la de-identificación fue bastante ineficaz dado que usando tan solo el zip es posible identificar a mas del 70% de la población registrada en el censo y para la edad, aunque se pudieron identificar a solo 2 personas, esto es igual de inseguro puesto que no cumple un mínimo de *2-anonimato*.

# Pregunta 3
El gobierno se da cuenta de que su intento de anonimización es muy vulnerable, por lo que solo permite a los analistas a realizar consultas de agregación sobre la tabla person que involucren muchas filas.
Obtenga el nivel educativo (education_num) del individuo llamado Ardyce Golby usando solo consultas de agregación que involucren al menos 1000 tuplas sobre la tabla *person*.

In [ ]:
educations = %sql select sum(education_num) as s from person
educations_without_ardyce = %sql select sum(education_num) as s from person where name != 'Ardyce Golby'
print("Nivel educacional de Ardyce Golby:", educations['s'].iloc[0] - educations_without_ardyce['s'].iloc[0])

 * postgresql+psycopg2://@/postgres
 * postgresql+psycopg2://@/postgres
Nivel educacional de Ardyce Golby: 12


#Pregunta 4


1.   Suponga ahora que un atacante cuenta con data auxiliar que contiene el cuasi-identificador código postal (zip). Indique qué nivel de $k$-Anonimato cumple person_deid.

Es *1-anonimato* ya que podemos identificar al menos a una persona con solo saber el cuasi identificador zip como se vio en la pregunta P2.1)

2.   Utilice alguna técnica de generalización y/o eliminación de valores atípicos sobre los cuasi-identificadores para lograr al menos $2$-anonimato.

In [ ]:
%%sql
SELECT (zip/50)*50 AS a,
count(*) AS c
FROM person
GROUP BY (zip/50)*50
ORDER BY c

 * postgresql+psycopg2://@/postgres


,a,c
0,25750,4
1,71100,5
2,3750,5
3,24200,5
4,16600,6
...,...,...
1995,3050,29
1996,41950,30
1997,49750,30
1998,32300,31


Con esto obtuvimos *4-anonimato*.

3.   Repita los pasos 1 y 2 para el CId edad (age).

In [ ]:
%%sql
SELECT LEAST((age/5)*5, 90) AS a,
count(*) AS c
FROM person
GROUP BY LEAST((age/5)*5, 90)
ORDER BY c

 * postgresql+psycopg2://@/postgres


,a,c
0,85,8
1,90,43
2,80,70
3,75,165
4,70,343
5,65,707
6,60,1308
7,15,1657
8,55,1864
9,50,2554


Obtuvimos *8-anonimato*

4.   Repita los pasos 1 y 2 para los CIds (zip, age). *Hint*: puede tratar de eliminar valores atípicos para la columna age.

In [ ]:
%%sql
SELECT LEAST((age/10)*10, 90) AS age, (zip/100)*100 AS zip,
count(*) AS cuenta
FROM person
GROUP BY LEAST((age/10)*10, 90), (zip/100)*100
HAVING count(age) > 1
ORDER BY cuenta

 * postgresql+psycopg2://@/postgres


,age,zip,cuenta
0,60,47400,2
1,10,3800,2
2,10,65200,2
3,60,93100,2
4,60,69300,2
...,...,...,...
5119,30,91000,18
5120,30,79200,18
5121,20,84800,19
5122,30,43100,20


5.   Concluya qué es lo que pasa cuando agregamos más CIds al análisis de $k$-anonimato.

Mientras agreguemos más CIds al análisis, los resultados son propensos a disminuir su *k-anonimato* ya que tenemos más información para realizar el cruce de las tablas, siendo así más fácil su identificación.

#Pregunta 5

Suponga que los CIds son edad y nivel de educación (age, education_num), y estado civil un dato sensible (marital_status).

1.   Escriba una consulta e indique si la tabla es $2$-diversa.

In [ ]:
%%sql
SELECT age, education_num, count(DISTINCT marital_status) AS c
FROM person GROUP BY (age, education_num)
ORDER BY c

 * postgresql+psycopg2://@/postgres


,age,education_num,c
0,66,15,1
1,66,2,1
2,66,1,1
3,65,12,1
4,65,8,1
...,...,...,...
960,27,9,7
961,38,9,7
962,26,9,7
963,30,13,7


No es *2-diversa*, ya que podemos ver que hay varias tuplas de edad y nivel de educación que tienen un solo estado marital.

2.   Utilice generalización y/o eliminación de valores atípicos para lograr al menos $5$-diversidad.


In [ ]:
%%sql
SELECT LEAST((age/17)*17, 68) AS age, (education_num/5)*5 AS education_num,
count(DISTINCT marital_status) AS cuenta
FROM person
GROUP BY LEAST((age/17)*17, 68), (education_num/5)*5
ORDER BY cuenta

 * postgresql+psycopg2://@/postgres


,age,education_num,cuenta
0,17,15,5
1,68,15,5
2,51,0,6
3,51,5,6
4,51,10,6
5,51,15,6
6,68,0,6
7,68,10,6
8,34,0,6
9,34,15,6


3.   Para el resultado en (2), escriba la consulta e indique el nivel de $k$-anonimato que se logra. ¿Qué puede concluir de esto?

In [ ]:
%%sql
SELECT LEAST((age/17)*17, 68) AS age, (education_num/5)*5 AS education_num,
count(*) AS cuenta
FROM person
GROUP BY LEAST((age/17)*17, 68), (education_num/5)*5
ORDER BY cuenta

 * postgresql+psycopg2://@/postgres


,age,education_num,cuenta
0,68,15,57
1,68,0,107
2,17,15,151
3,51,15,252
4,68,10,318
5,17,0,331
6,34,0,356
7,68,5,375
8,51,0,404
9,34,15,529


Mejorando la *l-diversidad* obtenemos mejores resultados para el *k-anonimato*, puesto que para cada grupo de filas con los mismos CIds ponemos una restricción adicional de que hayan al menos *l* datos sensibles distintos.

4.   (Opcional/Bonus) La **diversidad entrópica** de un grupo con el mismo CIds mide cuán uniformemente están distribuidos los valores privados de algún atributo $S$ en el grupo. Se define formalmente como
$$-\sum_{v\in S}\mathtt{P}(\mathit{cIds}, v)*\mathtt{log}(\mathtt{P}(\mathit{cIds}, v))$$ donde $\mathtt{P}(\mathit{cIds}, v)$ corresponde a la fracción de registros en el grupo $\mathit{cIds}$ con valor $v$ para el atributo $S$.
Por ejemplo, para la siguiente tabla donde la enfermedad es un dato sensible:

| Trabajo | Sexo | Edad | Enfermedad |
| --- | --- | --- | --- |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | Hepatitis |
| Profesor | Masculino | [35-40) | VIH |
| Artista | Femenino | [35-35) | Resfrío |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |
| Artista | Femenino | [35-35) | VIH |

La diversidad entrópica del grupo (Profesor, Masculino [35-40)) es de $-(\frac{2}{3}\mathtt{log}\frac{2}{3} + \frac{1}{3}\mathtt{log}\frac{1}{3}) = \mathtt{log}(1.9)$, y la diversidad entropica del grupo (Artista, Femenino, [35-35)) es $-(\frac{3}{4}\mathtt{log}\frac{3}{4} + \frac{1}{4}\mathtt{log}\frac{1}{4}) = \mathtt{log}(1.8)$.

Calcule la **diversidad entrópica** de cada grupo en (2), y concluya cuál es el conjunto de cids más y menos diversos.

Primero obtenemos los resultados de (2) incluyendo los estados maritales (no mostramos todos los conjuntos porque son varios pero cambiando las condiciones del HAVING se pueden verificar los resultados que obtuvimos):

In [ ]:
%%sql
SELECT LEAST((age/17)*17, 68) AS age, (education_num/5)*5 AS education_num,
marital_status, count(*)
FROM person
GROUP BY LEAST((age/17)*17, 68), (education_num/5)*5, marital_status
HAVING LEAST((age/17)*17, 68) = 68 AND (education_num/5)*5 = 15

 * postgresql+psycopg2://@/postgres


,age,education_num,marital_status,count
0,68,15,Divorced,3
1,68,15,Married-civ-spouse,46
2,68,15,Married-spouse-absent,1
3,68,15,Never-married,3
4,68,15,Widowed,4


De esto obtenemos las siguientes diversidades entrópicas:

$(17,5)=-(\frac{432}{5777}\mathtt{log}(\frac{432}{5777})+\frac{8}{5777}\mathtt{log}(\frac{8}{5777})+\frac{1542}{5777}\mathtt{log}(\frac{1542}{5777})+\frac{67}{5777}\mathtt{log}(\frac{67}{5777})+\frac{3521}{5777}\mathtt{log}(\frac{3521}{5777})+\frac{196}{5777}\mathtt{log}(\frac{196}{5777})+\frac{11}{5777}\mathtt{log}(\frac{11}{5777}))≈1.036$

$(17,10)=-(\frac{392}{6904}\mathtt{log}(\frac{392}{6904})+\frac{7}{6904}\mathtt{log}(\frac{7}{6904})+\frac{1786}{6904}\mathtt{log}(\frac{1786}{6904})+\frac{60}{6904}\mathtt{log}(\frac{60}{6904})+\frac{4526}{6904}\mathtt{log}(\frac{4526}{6904})+\frac{121}{6904}\mathtt{log}(\frac{121}{6904})+\frac{12}{6904}\mathtt{log}(\frac{12}{6904}))≈0.92$

$(17,15)=-(\frac{12}{151}\mathtt{log}(\frac{12}{151})+\frac{66}{151}\mathtt{log}(\frac{66}{151})+\frac{1}{151}\mathtt{log}(\frac{1}{151})+\frac{71}{151}\mathtt{log}(\frac{71}{151})+\frac{1}{151}\mathtt{log}(\frac{1}{151}))≈0.984$

$(34,5)=-(\frac{1062}{4917}\mathtt{log}(\frac{1062}{4917})+\frac{4}{4917}\mathtt{log}(\frac{4}{4917})+\frac{2675}{4917}\mathtt{log}(\frac{2675}{4917})+\frac{74}{4917}\mathtt{log}(\frac{74}{4917})+\frac{732}{4917}\mathtt{log}(\frac{732}{4917})+\frac{270}{4917}\mathtt{log}(\frac{270}{4917})+\frac{100}{4917}\mathtt{log}(\frac{100}{4917}))≈1.253$

$(34,10)=-(\frac{1435}{7136}\mathtt{log}(\frac{1435}{7136})+\frac{3}{7136}\mathtt{log}(\frac{3}{7136})+\frac{4173}{7136}\mathtt{log}(\frac{4173}{7136})+\frac{92}{7136}\mathtt{log}(\frac{92}{7136})+\frac{1112}{7136}\mathtt{log}(\frac{1112}{7136})+\frac{226}{7136}\mathtt{log}(\frac{226}{7136})+\frac{95}{7136}\mathtt{log}(\frac{95}{7136}))≈1.152$

$(34,15)=-(\frac{49}{529}\mathtt{log}(\frac{49}{529})+\frac{393}{529}\mathtt{log}(\frac{393}{529})+\frac{6}{529}\mathtt{log}(\frac{6}{529})+\frac{66}{529}\mathtt{log}(\frac{66}{529})+\frac{14}{529}\mathtt{log}(\frac{14}{529})+\frac{1}{529}\mathtt{log}(\frac{1}{529}))≈0.86$

$(51,5)=-(\frac{446}{2487}\mathtt{log}(\frac{446}{2487})+\frac{1493}{2487}\mathtt{log}(\frac{1493}{2487})+\frac{27}{2487}\mathtt{log}(\frac{27}{2487})+\frac{148}{2487}\mathtt{log}(\frac{148}{2487})+\frac{81}{2487}\mathtt{log}(\frac{81}{2487})+\frac{292}{2487}\mathtt{log}(\frac{292}{2487}))≈1.195$

$(51,10)=-(\frac{431}{2460}\mathtt{log}(\frac{431}{2460})+\frac{1611}{2460}\mathtt{log}(\frac{1611}{2460})+\frac{28}{2460}\mathtt{log}(\frac{28}{2460})+\frac{160}{2460}\mathtt{log}(\frac{160}{2460})+\frac{56}{2460}\mathtt{log}(\frac{56}{2460})+\frac{174}{2460}\mathtt{log}(\frac{174}{2460}))≈1.085$

$(51,15)=-(\frac{24}{252}\mathtt{log}(\frac{24}{252})+\frac{193}{252}\mathtt{log}(\frac{193}{252})+\frac{2}{252}\mathtt{log}(\frac{2}{252})+\frac{26}{252}\mathtt{log}(\frac{26}{252})+\frac{1}{252}\mathtt{log}(\frac{1}{252})+\frac{6}{252}\mathtt{log}(\frac{6}{252}))≈0.812$

$(68,5)=-(\frac{26}{375}\mathtt{log}(\frac{26}{375})+\frac{1}{375}\mathtt{log}(\frac{1}{375})+\frac{198}{375}\mathtt{log}(\frac{198}{375})+\frac{4}{375}\mathtt{log}(\frac{4}{375})+\frac{22}{375}\mathtt{log}(\frac{22}{375})+\frac{3}{375}\mathtt{log}(\frac{3}{375})+\frac{121}{375}\mathtt{log}(\frac{121}{375}))≈1.156$

$(68,10)=-(\frac{27}{318}\mathtt{log}(\frac{27}{318})+\frac{168}{318}\mathtt{log}(\frac{168}{318})+\frac{6}{318}\mathtt{log}(\frac{6}{318})+\frac{33}{318}\mathtt{log}(\frac{33}{318})+\frac{6}{318}\mathtt{log}(\frac{6}{318})+\frac{78}{318}\mathtt{log}(\frac{78}{318}))≈1.276$

$(68,15)=-(\frac{3}{57}\mathtt{log}(\frac{3}{57})+\frac{46}{57}\mathtt{log}(\frac{46}{57})+\frac{1}{57}\mathtt{log}(\frac{1}{57})+\frac{3}{57}\mathtt{log}(\frac{3}{57})+\frac{4}{57}\mathtt{log}(\frac{4}{57}))≈0.74$

De esto podemos concluir que el grupo de cids más diverso es (68, 10), mientras que el menos diverso es (68, 15).

# Pregunta 6

1.   Escriba una consulta que responda a "¿cuántas personas nunca se han casado?"

In [ ]:
%%sql
SELECT count(*) AS cuenta
FROM person
GROUP BY marital_status
HAVING marital_status = 'Never-married'

 * postgresql+psycopg2://@/postgres


,cuenta
0,10683


2.   La sensibilidad de una consulta se puede calcular como: cuánto es lo máximo que puede variar el resultado de mi consulta si saco/agrego un único individuo de la base de datos. Escriba y justifique la sensibilidad de la consulta de (1).

Para esta consulta lo máximo que puede variar el resultado es en 1, dado que si agregamos o eliminamos a una persona de la base de datos, si justo se daba el caso de que nunca se ha casado entonces el resultado de esta consulta va a variar en tan sólo 1.

3.   Use la distribución de laplace para contestar (1) de manera diferencialmente privada, con epsilon = 0.1.
    
    *Hint*: Para generar ruido aleatorio puede utilizar la función laplace de numpy:
```python
import numpy as np
np.random.laplace(loc=0, scale=s)
```
y para aplicar una función a todos los elementos de un resultado de un consulta (dataframe) puede usar la función apply:
```python
r = %sql ...
r.apply(lambda x: ...)
```

In [ ]:
import numpy as np
r = %sql SELECT count(*) AS cuenta FROM person GROUP BY marital_status HAVING marital_status = 'Never-married'
r.apply(lambda x: x + np.random.laplace(loc=0, scale=1/0.1))

 * postgresql+psycopg2://@/postgres


,cuenta
0,10669.850575


4.   Escriba una consulta $0.1$-diferencialmente privada que calcule la cantidad de personas agrupadas por edad y estado civil. Asegúrese que su respuesta no contenga cantidades negativas ni decimales (puede modificar los valores después de agregarle ruido).

    *Hint*: Use la función map para aplicar una función a todos los elementos de una columna de la siguiente forma:
```python
r = %sql ...
r['nombre_columna_nueva'] = r['nombre_columna'].map(lambda x: ...)
```

In [ ]:
r = %sql SELECT age, marital_status, count(*) AS cuenta FROM person GROUP BY (age, marital_status)
r['cuenta'] = r['cuenta'].map(lambda x: x + np.random.laplace(loc=0, scale=1/0.1)).map(lambda x: 0 if x < 0 else int(round(x, 0)))
r

 * postgresql+psycopg2://@/postgres


,age,marital_status,cuenta
0,44,Never-married,82
1,60,Widowed,29
2,20,Married-spouse-absent,11
3,42,Divorced,182
4,49,Separated,40
...,...,...,...
391,82,Married-civ-spouse,0
392,73,Married-spouse-absent,2
393,37,Never-married,167
394,82,Never-married,0


In [ ]:
#Consulta sin ruido, hecha solo para comparar la consulta con y sin ruido
r = %sql SELECT age, marital_status, count(*) AS cuenta FROM person GROUP BY (age, marital_status)
r['cuenta']= r['cuenta'].map(lambda x: 0 if x < 0 else int(round(x, 0)))
r

 * postgresql+psycopg2://@/postgres


,age,marital_status,cuenta
0,44,Never-married,93
1,60,Widowed,25
2,20,Married-spouse-absent,5
3,42,Divorced,173
4,49,Separated,20
...,...,...,...
391,82,Married-civ-spouse,4
392,73,Married-spouse-absent,1
393,37,Never-married,158
394,82,Never-married,1


5.    Comente acerca de la utilidad y la privacidad de la respuesta para grupos muy chicos vs grupos grandes después de agregar ruido.

Independientemente de si se trata de un grupo pequeño o grande, la adición de ruido genera respuestas más privadas. Este enfoque ofrece la ventaja de poder atribuir cualquier resultado concluyente al ruido introducido. Específicamente, en grupos pequeños, la incorporación de ruido puede tener un impacto más significativo en la utilidad y precisión de los resultados. Sin embargo, este efecto se contrarresta con la protección que brinda a los individuos contra la posibilidad de que se revelen sus datos específicos, lo que resulta crucial para preservar la privacidad.

En grupos más grandes, la adición de ruido puede tener un impacto menor en la utilidad y precisión de los resultados. Aunque la identificación individual puede ser menos probable en estos casos, persiste el riesgo potencial de revelar información sensible. Por lo tanto, sigue siendo una consideración esencial para proteger la información individual, incluso cuando se trabaja con conjuntos de datos más extensos.